In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

def get_challenger_name():
    
    driver = webdriver.Chrome()

    driver.get("https://www.op.gg/leaderboards/tier")

    tmp_player = driver.find_elements(By.CLASS_NAME,"summoner-link")

    summoner_list = []

    for x in tmp_player:
        summoner_list.append(x.get_attribute("href"))

    driver.quit()
    return summoner_list

summoner_list = get_challenger_name()

try:
    pd.DataFrame(summoner_list).to_csv("data/summoner_list.csv",index=False,header=False)
except BaseException:
    print("Error is ",BaseException)


In [ ]:
# The next phase is to open each challenger name
# Then get 100 game from them
# To do that we will have to click the READ MORE button

def collect_hundred_games(summoner_link):
    driver = webdriver.Chrome()

    driver.get(summoner_link+"?queue_type=SOLORANKED")

    # first click the agree button

    time.sleep(15)

    # Now click the agree button

    agree_button = driver.find_element(By.CLASS_NAME, "css-47sehv")
    agree_button.click()
    print("<==========  Agree button clicked  ================>")


    expand_btn = driver.find_elements(By.CLASS_NAME, "more")

    for i in range(4):
            for x in expand_btn:
                if(x.tag_name == "button"):
                    time.sleep(5)
                    x.click()
                    print("*",sep=" ",end="")
                    time.sleep(5)

    print("")

    match_list = driver.find_elements(By.CLASS_NAME, "ery81n90")

    result = []

    percentage = 0
    print(" Loading the URLs")

    print(len(match_list))

    try:
        if(len(match_list) >= 100):
             for x in match_list:
                #   Function to click a button then copy the link text
                # Then append the button link to a list
                game_link_btn = x.find_element(By.CLASS_NAME, "btn-detail")
                game_link_btn.click()
                time.sleep(0.1)
                # After clicking the button we need to fetch the game link
                # Then append the link to a list
                game_url = x.find_element(By.CLASS_NAME, "copy-link")
                print(game_url.get_attribute("value"), "<=== ",percentage," ===>")
                percentage+=1
                result.append(game_url.get_attribute("value"))
    except BaseException:
        print("Error = ", BaseException)
        driver.quit()
    
    driver.quit()
    return result

# Now iterate and save the CSV files
count = 0
for x in summoner_list:
    result = collect_hundred_games(x)
    df = pd.DataFrame(result)

    df.to_csv(f'data/scrap_hero_{count}.csv',index=False,header=False)
    count+=1
    print(" ============ ", count,"%  ============")

In [ ]:
import os
import pandas as pd
import time

game_cards = os.listdir("data/")

sunfire_archive = []


def get_list_from_df(df):
    path = "data/"+df
    sln = []
    try:
        result = pd.read_csv(path)

        # iterate over each rows
        # for x in result:
        #     sln.append(x)
        for row in result.iterrows():
            sln.append(row[1].values)
            # print(row[1].values)
    except BaseException as e:
        print(e,"==>",df)
        sln = [""]

    return sln

for x in game_cards:
    ite = get_list_from_df(x)
    for game_id in ite:
        sunfire_archive.append(game_id)

final_df = pd.DataFrame(sunfire_archive)

final_df.to_csv("Shurima_Bibliothek.csv",header=False,index=False)